<a href="https://colab.research.google.com/github/LilyRosa/Matematica-Numerica-Google-Colab/blob/main/notebooks/cap3/Jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import array, absolute, copy, zeros
import pandas as pd

# Algoritmo de Jacobi

Algoritmo utilizado para dar solución a sistemas lineales de orden $n$ en la forma $AX = B$ con error menor que $\varepsilon$


## Implementación

### Clase Auxiliar de Resultado de Jacobi

Clase auxiliar usada para guardar los resultados de cada iteración del algoritmo de Jacobi

In [2]:
class ResultadoJacobi:
    def __init__(self):
        self.lista_x = []
        self.delta = 0
        self.error = 0
        self.iteracion_0 = False

### Algoritmo de Jacobi


` jacobi(a, b, x0, f_convergencia, tol, max_iter): ` Implementación del algoritmo de Jacobi para dar solución a sistemas de ecuaciones lineales

##### Parámetros

- ` a ` : matriz de los coeficientes $A$
- ` b ` : matriz de los términos independientes $B$
- ` x0 ` : matriz columna que representa los valores estimados de solución (se puede utilizar la matriz trivial)
- ` f_convergencia ` : define el factor de convergencia de la matriz $A$
- ` tol ` : cota para el error absoluto
- ` max_iter `: cantidad máxima de iteraciones

In [3]:
def jacobi(a, b, x0, f_convergencia, tol, max_iter):
    xv = copy(x0)
    xa = zeros(b.shape[0])
    condicion = True
    paso = 1
    retorno = []
    r = ResultadoJacobi()

    r.iteracion_0 = True
    r.lista_x = xv.tolist()
    retorno.append(r)

    while condicion:
        r = ResultadoJacobi()
        error = 0

        for i in range(a.shape[0]):
            xa[i] = b[i]

            for j in range(a.shape[1]):
                if j != i:
                    xa[i] -= a[i][j] * xv[j]

            xa[i] /= a[i][i]

            if abs(xa[i] - xv[i]) > error:
                error = abs(xa[i] - xv[i])

        r.delta = error
        error *= abs(f_convergencia / (1 - f_convergencia))
        r.error = error
        r.lista_x = xa.tolist()
        retorno.append(r)

        xv = copy(xa)
        paso += 1
        condicion = error > tol and paso <= max_iter

    return retorno

### Hallar factor de convergencia $\alpha$

` hallar_factor_convergencia(a): ` Halla el factor de convergencia $\alpha$ de la matriz $A$

##### Parámetros
- ` a ` : matriz de los coeficientes $A$

In [4]:
def hallar_factor_convergencia(a):

    a = absolute(a)
    resultado = []

    for i in range(len(a)):
        total_fila = sum(a[i])

        if total_fila - a[i][i] < 0:
            raise Exception("El factor de convergencia de la matriz tiene que ser mayor que 0")

        resultado.append((total_fila - a[i][i]) / a[i][i])

    return max(resultado)

### Métodos Auxiliares

In [5]:
def hallar_f_convergencia_error(f_convergencia):
    return abs(f_convergencia / (1 - f_convergencia))


def convertir_headers_resultados(lista_resultados_jacobi):
    lista = []

    r = lista_resultados_jacobi[0]
    for i in range(len(r.lista_x)):
        lista.append(f'X{i + 1}')
    lista.append('Delta (δ)')
    lista.append('Error')

    return lista


def convertir_resultados(lista_resultados_jacobi):
    lista = []
    for r in lista_resultados_jacobi:
        l = []
        if r.iteracion_0:
            for x in r.lista_x:
                l.append(x)
            l.append('-------')
            l.append('-------')
        else:
            for x in r.lista_x:
                l.append(x)
            l.append('{:.7f}'.format(r.delta))
            l.append('{:.7f}'.format(r.error))

        lista.append(l)

    df = pd.DataFrame(data=lista, columns=convertir_headers_resultados(lista_resultados_jacobi))
    df.index.name = 'Iteración'
    return df

## Inserción de datos

#### Datos

- ` a ` : matriz de los coeficientes $A$
- ` b ` : matriz de los términos independientes $B$
- ` x0 ` : matriz columna que representa los valores estimados de solución (se puede utilizar la matriz trivial)
- ` tol ` : cota para el error absoluto
- ` max_iter `: cantidad máxima de iteraciones

In [6]:
a = array([[5, -1, 1],
           [2, 5, -1],
           [-1, 1, 5]])

b = array([10, 12, 10])

x0 = array([0, 0, 0])

tol = 0.005

max_iter = 100

## Salida de datos

### Factor de convergencia $\alpha$

In [7]:
print(f'El factor de convergencia es {hallar_factor_convergencia(a)}')
print('ɑ/(1-ɑ) = {:.5f}\n'.format(hallar_f_convergencia_error(hallar_factor_convergencia(a))))

El factor de convergencia es 0.6
ɑ/(1-ɑ) = 1.50000


### Resultados de la aplicación del método de Jacobi

In [8]:
r = jacobi(a, b, x0, hallar_factor_convergencia(a), tol, max_iter)
df = convertir_resultados(r)
df

,X1,X2,X3,Delta (δ),Error
Iteración,,,,,
0,0.000000,0.000000,0.000000,-------,-------
1,2.000000,2.400000,2.000000,2.4000000,3.6000000
2,2.080000,2.000000,1.920000,0.4000000,0.6000000
3,2.016000,1.952000,2.016000,0.0960000,0.1440000
4,1.987200,1.996800,2.012800,0.0448000,0.0672000
5,1.996800,2.007680,1.998080,0.0147200,0.0220800
6,2.001920,2.000896,1.997824,0.0067840,0.0101760
7,2.000614,1.998797,2.000205,0.0023808,0.0035712
